In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
spark.version

'3.3.2'

In [7]:
!ls -lh fhvhv_tripdata_2021-06.csv

-rw-r--r-- 1 jriva 197609 878M Dec 14 21:00 fhvhv_tripdata_2021-06.csv


In [8]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [12]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/', mode='overwrite')

In [13]:
df = spark.read.parquet('data/pq/fhvhv/2021/06/')

**Q3**: How many taxi trips were there on February 15?

In [14]:
from pyspark.sql import functions as F

In [19]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

450872

In [20]:
df.registerTempTable('fhvhv_2021_06')

In [21]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  450872|
+--------+



**Q4**: Longest trip for each day

In [22]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [23]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-22|         null|
| 2021-06-04|         null|
| 2021-06-20|         null|
| 2021-06-27|         null|
| 2021-06-28|         null|
+-----------+-------------+



In [24]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60) AS duration
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+--------+
|pickup_date|duration|
+-----------+--------+
| 2021-06-22|    null|
| 2021-06-04|    null|
| 2021-06-20|    null|
| 2021-06-27|    null|
| 2021-06-28|    null|
| 2021-06-01|    null|
| 2021-06-17|    null|
| 2021-06-13|    null|
| 2021-06-19|    null|
| 2021-07-01|    null|
+-----------+--------+



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [25]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
| 2021-06-05 19:15:59|      28|
| 2021-06-11 19:32:39|      26|
| 2021-06-12 21:43:40|      26|
| 2021-06-12 22:37:37|      25|
| 2021-06-11 18:01:44|      25|
+--------------------+--------+



In [26]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-----+
|dispatching_base_num|count|
+--------------------+-----+
| 2021-06-05 19:15:59|   28|
| 2021-06-11 19:32:39|   26|
| 2021-06-12 21:43:40|   26|
| 2021-06-12 22:37:37|   25|
| 2021-06-11 18:01:44|   25|
+--------------------+-----+



**Q6**: Most common locations pair

In [27]:
df_zones = spark.read.parquet('zones')

In [28]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [29]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [30]:
df_zones.registerTempTable('zones')

In [31]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+----------+--------+
|pu_do_pair|count(1)|
+----------+--------+
|      null|14961892|
+----------+--------+

